In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow
import sys
import os
sys.path.append(os.path.abspath('..'))

from src.utils import *
from src.features import *
from src.models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
train = pd.read_parquet('../data/train.parquet')
print("Train shape:", train.shape)
test = pd.read_parquet('../data/test.parquet')
print("Test shape:", test.shape)

Train shape: (18145372, 126)
Test shape: (6897776, 125)


In [4]:
train.head()

,Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,...,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected
0,0,True,57323,<NA>,S7/SU/UT,36,False,False,2024-06-15T16:20:00,2024-06-15T15:40:00,...,1.0,1,2087645,98ce0dabf6964640b63079fbafd42cbe,2024-05-17 03:03:08,TLKKJA/KJATLK,True,370.0,16884.0,1
1,1,True,57323,123,S7/SU/UT,36,True,False,2024-06-15T14:50:00,2024-06-15T09:25:00,...,1.0,1,2087645,98ce0dabf6964640b63079fbafd42cbe,2024-05-17 03:03:08,TLKKJA/KJATLK,True,2240.0,51125.0,0
2,2,True,57323,<NA>,S7/SU/UT,36,False,False,2024-06-15T14:50:00,2024-06-15T09:25:00,...,1.0,1,2087645,98ce0dabf6964640b63079fbafd42cbe,2024-05-17 03:03:08,TLKKJA/KJATLK,True,2240.0,53695.0,0
3,3,True,57323,123,S7/SU/UT,36,True,False,2024-06-15T14:50:00,2024-06-15T09:25:00,...,1.0,1,2087645,98ce0dabf6964640b63079fbafd42cbe,2024-05-17 03:03:08,TLKKJA/KJATLK,True,2240.0,81880.0,0
4,4,True,57323,<NA>,S7/SU/UT,36,False,False,2024-06-15T14:50:00,2024-06-15T09:25:00,...,1.0,1,2087645,98ce0dabf6964640b63079fbafd42cbe,2024-05-17 03:03:08,TLKKJA/KJATLK,True,2240.0,86070.0,0


In [ ]:
train[['Id', 'profileId', 'ranker__id', '

In [16]:
print('=== Basic EDA ===')
print(f"Total Users: {train['profileId'].nunique():,}")
print(f"Total Items: {train['Id'].nunique():,}")
print(f"Total Sessions: {train['ranker_id'].nunique():,}")
# 한번의 검색에서 사용자가 제시받는 항공편 옵션들 = 1 session
print(f"Select Rate: {train['selected'].mean():.4f}")

# 세션당 후보 수 분포
session_sizes = train.groupby('ranker_id').size()
# ranker_id(1개의 세션)의 크기 -> 몇 개의 항공편이 제시되었는지 확인
print(f"\nPer Session: {session_sizes.describe()}")

=== Basic EDA ===
Total Users: 32,922
Total Items: 18,145,372
Total Sessions: 105,539
Select Rate: 0.0058

Per Session: count    105539.000000
mean        171.930490
std         445.940118
min           1.000000
25%          19.000000
50%          50.000000
75%         154.000000
max        8236.000000
dtype: float64


In [19]:
session_stats = train.groupby('ranker_id')['selected'].agg(['sum', 'count'])
sessions_with_selection = (session_stats['sum'] > 0).sum()
print(f"실제 예약으로 이어진 세션 비율: {sessions_with_selection/len(session_stats):.4f}")

실제 예약으로 이어진 세션 비율: 1.0000
